In [14]:
!pip install pandas
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4

# Imports

In [37]:
#HTMl
from bs4 import BeautifulSoup
import html5lib
import requests

#Calcs
import pandas as pd
import numpy as np

#Graphs
from bokeh.charts import Bar, Donut, output_notebook, show
from bokeh.layouts import gridplot, row
from bokeh.models import HoverTool
from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import figure

In [2]:
url = "http://www.transparencia.gov.br/PortalComprasDiretasOEUnidadeGestora.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243"

response = requests.get(url)

content = response.content

soup = BeautifulSoup(content, "html.parser")

In [3]:
a_tags = soup.find_all("a")
cont = 0;
for link in a_tags:
    if "PortalComprasDiretasOEElementoDespesa" in link.get("href"):
        print(link.get("href"))
    cont += 1

print(cont)

PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153109
PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153107
PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153253
PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153113
PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153977
PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153103
23


# Funções de tratamentos dos dados

In [4]:
####IVAN

#Funções para tratamento das strings
#Remove o "." dos numeros(strings) na tabela e corrige os espaços
#causados por diferença de codificação ( \xa0 -> ' ')
def fix_str(val):
    for char in val:
        if "." in char:
            val = val.replace(char,'')
        elif "\xa0" in char:
            val = val.replace(char,' ')
    return val
#Converte a string da tabela para um float e adiciona as duas casas
#decimais após a virgula.
def fix_decimal_n_float(val):
    return float(val)/100

#lê URL, cria dataframe do pandas. Essa função cria um array de dataframes. Logo, se um site
#possui mais de uma tabela, elas serão agrupadas em forma de array.
def retorna_tabelas(val):
    return pd.read_html(val)

#Percorre a tabela e converte o string para float
#Passe como parâmetro dessa função a tabela que você quer normalizar e o nome da coluna
#que será convertida para float.
def corrige_tabela(table, column_name):
    for atual in range(len(table)):
        table.iloc[atual][column_name] = fix_decimal_n_float(fix_str(
                                        table.iloc[atual][column_name]))
        
#Cria a tabela a partir do DataFrame lido do HTML
#Passe como entrada o dataframe e a função retornará a tabela pronta, com as colunas
#nomeadas.
def cria_tabela(df, inicio=0, fim=0):
    col_vet = []
    #se o range de copia não é definido, o função percorre até
    #o final da tabela
    if fim == 0 :
        fim = len(df)
        
    #Nomeia as tabelas com numero de colunas variado
    for atual in range(0, df.columns.values.size):
        col_vet.append(fix_str(df[atual][0]))  
            
    
    temp = df[inicio:fim].copy()
    temp.columns = col_vet
    return temp

####IVAN

In [5]:
####IVAN
dfs = retorna_tabelas(url)

#Tabela com todo o dinheiro distribuído para a educação e quanto é
#destinado a UFRN
# não dá para usar o cria tabela, pois os valores 'numericos', já são float MUITO ESTRANHO
Gastos_totais = dfs[0][1:].copy()
Gastos_totais.columns = [dfs[0][0][0], "Moeda", dfs[0][1][0]]

#Tabela com o dinheiro da UFRN e como ele é distribuído entre as unidades
Gastos_por_unidade = cria_tabela(dfs[1], 1)
Gastos_por_unidade.drop("Código", axis=1, inplace=True)

#Percorre a tabela e converte o string para float
#O e+ é uma reação padrão da linguagem, para que os valores caibam na tabela
#Ao acessar cada valor individualmente, é possivel pegar o valor correto e completo
corrige_tabela(Gastos_por_unidade, "Total no Ano (R$)")

####IVAN

In [6]:
Gastos_por_unidade

,Unidade Gestora,Total no Ano (R$)
1,CENTRO DE BIOCIENCIAS DA UFRN,2.18907e+06
2,CENTRO DE CIENCIAS DA SAUDE DA UFRN,3.73863e+06
3,CENTRO DE CIENCIAS HUM.E LETRAS ARTES DA UFRN,2.56673e+06
4,CENTRO DE CIENCIAS SOCIAIS APLICADAS DA UFRN,1.26147e+06
5,CENTRO DE ENSINO SUPERIOR DO SERIDO/UFRN,731373
6,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,1.40942e+09


# Plotagem

In [7]:
####IVAN


#Verificar se esse gráfico é mesmo necessário e solucionar o problema
# da diferença de valores
graph = Donut(Gastos_por_unidade[:5], label="Unidade Gestora", values="Total no Ano (R$)", title="Distribuição do dinheiro" )
graph.add_tools(HoverTool(tooltips=[("Unidade:", "@{Unidade Gestora}"),
                                    ("Total:", "@{Total no Ano (R$)}")
                                   ]
                         ))

#Diferença de valores muito grande. :/
graph2 = Bar(Gastos_por_unidade, values="Total no Ano (R$)", label="Unidade Gestora", color="Unidade Gestora", title="Distribuição do dinheiro")


layout = gridplot([[graph, None], [None, graph2]], sizing_mode='scale_width')
output_notebook()
show(layout)

####IVAN

Loading BokehJS ...

### Pega o site

In [38]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from copy import copy
import re    

class PortalTransparencia():
    def __init__(self):
        self.main_url = 'http://www.transparencia.gov.br/PortalComprasDiretasOEUnidadeGestora.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243'

        self.queue_next_urls = [] # urls to run into the crawler
        
        self.data_collected = []
        
        self.final_dictonary = {'centro': None, 'link': None}

    def stop_crawler(self):
        self.continue_crawler = False

    def run(self):
        html = self.getBody(self.main_url)
        urls = self.get_centros_url(html)
        for dictionary in urls:
            url = dictionary['link']
            temp_url = 'http://www.transparencia.gov.br/' + url
            html = self.getBody(temp_url)
            sub_links = self.get_subcentros_url(html)
            for link in sub_links:
                temp_suburl = 'http://www.transparencia.gov.br/' + link
                html = self.getBody(temp_suburl)
                self.getData(dictionary['centro'], temp_suburl, html)

    def getBody(self, url):
        res = urlopen(url)
        return BeautifulSoup(res.read())

    def get_centros_url(self, html):
        urls = []
        link_tags = html.findAll('a')
        for link in link_tags:
            if link.string != None and 'CENTRO' in link.string:
                temp_dic = copy(self.final_dictonary)
                temp_dic['centro'] = link.string
                temp_dic['link'] = link['href']
                urls.append(temp_dic)
        return urls
    
    def get_subcentros_url(self, html):
        links = []
        table_tags = html.findAll('table')
        a_tags = table_tags[1].findAll('a')
        for link in a_tags:
            if '-' in link.string:
                links.append(link['href'])
            #if 'Investimentos' in link.string:
            #    links.append(link['href'])
            #elif 'Outras' in link.string:
            #    links.append(link['href'])
        return links    
        
    def getData(self, centro, link, html):
        final_data = []
        tables = html.findAll('table')
        if len(tables) > 0:
            data = copy(self.final_dictonary)
            data['centro'] = centro
            data['link'] = link
            self.data_collected.append(data)

In [ ]:
####IVAN

##Percorre os links e cria tabelas para cada um deles

### Aqui que entra o SCRAP
urls = []
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153109")
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153107")
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153253")
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153113")
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153977")
urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEElementoDespesa.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153103")

##Lista de tabelas
Departamentos = []
#Criar um dicionário da seguinte form dict = { "Nome_Do_departamento" : Tabela}, assim fica mais fácil de 
#identificar cada gráfico que será plotado
for atual in range(len(urls)):
    df = retorna_tabelas(urls[atual])
    Departamentos.append(cria_tabela(df[1], 1))

####IVAN

In [9]:
####IVAN
for atual in range(len(Departamentos)):
    corrige_tabela(Departamentos[atual], "Total no Ano (R$)" )
    
####IVAN

In [10]:
Graficos = []

for atual in range(len(Departamentos)):
    graph = Bar(Departamentos[atual], values="Total no Ano (R$)", label="Elemento de Despesa", color="Elemento de Despesa",
                title=str(atual)+" Inisa o nome da UNIDADE", legend=None)
    graph.add_tools(HoverTool(tooltips=[("Gasto", "$y{f.ff}")]))
    Graficos.append(graph)

grfs = [Graficos[0:2], Graficos[2:4], Graficos[4:6]]

layout = gridplot(grfs, sizing_mode='scale_width')
output_notebook()

show(layout)

Loading BokehJS ...

# Teste do scatterplot

In [11]:
url ="http://www.transparencia.gov.br/PortalComprasDiretasOEFavorecidoED.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153109&CodigoGD=4&CodigoED=52"
url2 ="http://www.transparencia.gov.br/PortalComprasDiretasOEFavorecidoED.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243&CodigoUG=153107&CodigoGD=4&CodigoED=52"
df = retorna_tabelas(url)
df2 = retorna_tabelas(url2)

In [12]:
tb = cria_tabela(df[1], 1)
tb = tb.append(cria_tabela(df2[1], 1))

tb.reset_index(inplace=True)
tb.drop("index", axis=1, inplace=True)

tb.sort_values("Favorecido [Nome Fantasia]", inplace=True)

In [13]:
corrige_tabela(tb, "Total no Ano (R$)")
#new_tb = tb.groupby(['CPF/CNPJ/Outros', 'Favorecido [Nome Fantasia]'])['Total no Ano (R$)'].transform('sum')
new_tb = tb.reset_index().groupby(['CPF/CNPJ/Outros', 'Favorecido [Nome Fantasia]'])['Total no Ano (R$)'].sum()
new_tb = new_tb.to_frame()

In [14]:
print(new_tb.columns.values)
new_tb['Total no Ano (R$)']

#Criar gráfico de compras dos departamentos da UF

['Total no Ano (R$)']


CPF/CNPJ/Outros     Favorecido [Nome Fantasia]                                                                                                
01.044.210/0001-44  AMCANAA EQUIPAMENTOS INDUSTRIAIS LTDA - EPP                                                                                     580.00
03.213.418/0001-75  BUNKER COMERCIAL LTDA - EPP                                                                                                   85895.00
04.129.107/0001-95  FENIX INDUSTRIA E COMERCIO DE MOVEIS LTDA - ME [FENIX]                                                                         9040.00
06.198.597/0001-07  APFORM INDUSTRIA E COMERCIO DE MOVEIS LTDA                                                                                    10009.44
06.936.105/0001-33  BIOVERA EQUIPAMENTOS E SERVICOS LTDA - ME                                                                                       750.00
07.179.175/0001-57  DATASONIC INDUSTRIA E DISTRIBUICAO DE ELETRONICOS LTDA [DATASO

# TESTES

In [106]:
#Lines distribuction

url = "http://www.transparencia.gov.br/PortalComprasDiretasOEUnidadeGestora.asp?Ano=2016&CodigoOS=26000&CodigoOrgao=26243"

####IVAN

##Percorre os links e cria tabelas para cada um deles

### Aqui que entra o SCRAP
urls.clear()

for atual in range(10,18):
    urls.append("http://www.transparencia.gov.br/PortalComprasDiretasOEUnidadeGestora.asp?Ano=20"+str(atual)+"&CodigoOS=26000&CodigoOrgao=26243")

##Lista de tabelas
Departamentos = []
#Criar um dicionário da seguinte form dict = { "Nome_Do_departamento" : Tabela}, assim fica mais fácil de 
#identificar cada gráfico que será plotado
for atual in range(len(urls)):
    df = retorna_tabelas(urls[atual])
    Departamentos.append(cria_tabela(df[1], 1))
    
for atual in range(0,len(Departamentos)):
    Departamentos[atual]["Ano"] = atual+2010
    
####IVAN


In [117]:
#for atual in range(0,len(Departamentos)):
#    corrige_tabela(Departamentos[atual], "Total no Ano (R$)" )

temp = Departamentos[0].copy()
corrige_tabela(temp, "Total no Ano (R$)" )
#for atual in range(0,len(Departamentos)):
#    print(type(Departamentos[atual]["Total no Ano (R$)"][1] ))

C:\Users\ivana\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [118]:
temp

,Código,Unidade Gestora,Total no Ano (R$),Ano
1,153109,CENTRO DE BIOCIENCIAS DA UFRN,1.444.01121,2010
2,153107,CENTRO DE CIENCIAS DA SAUDE DA UFRN,4.163.27096,2010
3,153111,CENTRO DE CIENCIAS EXATAS E DA TERRA DA UFRN,115.80663,2010
4,153253,CENTRO DE CIENCIAS HUM.E LETRAS ARTES DA UFRN,2.147.15052,2010
5,153113,CENTRO DE CIENCIAS SOCIAIS APLICADAS DA UFRN,1.711.91415,2010
6,153977,CENTRO DE ENSINO SUPERIOR DO SERIDO/UFRN,536.68592,2010
7,153112,CENTRO DE TECNOLOGIA DA UFRN,867.13914,2010
8,153252,HOSP.PEDIATRIA PROF. HERIBERTO F. BEZERRA,2.657.44220,2010
9,153104,HOSPITAL UNIVERSITARIO ANA BEZERRA,2.812.32206,2010
10,153108,HOSPITAL UNIVERSITARIO ONOFRE LOPES DA UFRN,21.085.82302,2010


In [88]:
#Lines por departamento

dep_lines = figure(title="Verbas por departamento", x_axis_type="datetime")

dep_lines.line(Departamentos["Ano"], Departamentos["Total no Ano (R$)"], legend="y=sqrt(x)", line_color="tomato", line_dash="dotdash")

output_notebook()

show(dep_lines)


TypeError: list indices must be integers or slices, not str

str

In [89]:
Departamentos

[    Código                                Unidade Gestora Total no Ano (R$)  \
 1   153109                  CENTRO DE BIOCIENCIAS DA UFRN       1.444.01121   
 2   153107            CENTRO DE CIENCIAS DA SAUDE DA UFRN       4.163.27096   
 3   153111   CENTRO DE CIENCIAS EXATAS E DA TERRA DA UFRN         115.80663   
 4   153253  CENTRO DE CIENCIAS HUM.E LETRAS ARTES DA UFRN       2.147.15052   
 5   153113   CENTRO DE CIENCIAS SOCIAIS APLICADAS DA UFRN       1.711.91415   
 6   153977       CENTRO DE ENSINO SUPERIOR DO SERIDO/UFRN         536.68592   
 7   153112                   CENTRO DE TECNOLOGIA DA UFRN         867.13914   
 8   153252      HOSP.PEDIATRIA PROF. HERIBERTO F. BEZERRA       2.657.44220   
 9   153104             HOSPITAL UNIVERSITARIO ANA BEZERRA       2.812.32206   
 10  153108    HOSPITAL UNIVERSITARIO ONOFRE LOPES DA UFRN      21.085.82302   
 11  153110      MATERNIDADE ESCOLA JANUARIO CICCO DA UFRN      10.148.07279   
 12  153103    UNIVERSIDADE FEDERAL DO R